In [49]:
import configparser
import time
import numpy as np
import os
import tensorflow as tf
import configparser
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import importlib
import utils

importlib.reload(utils)

import generator
importlib.reload(generator)

import GNN
importlib.reload(GNN)

params = configparser.ConfigParser()
params._interpolation = configparser.ExtendedInterpolation()
params.read('./config.ini')

['./config.ini']

In [50]:

# directory = 'DIRECTORIES_' + params['RUN_CONFIG']['dataset']
directory = 'DIRECTORIES_' + 'IDS2017_LABELS_IDS2017'
# (model, startingEpoch) = make_or_restore_model(params=params, forceRestore='weights.17-0.02.tf')
model = GNN.GNN(params, loadBestEpoch=True)


Model not in the GPU.


wandb: WARNING `resume` will be ignored since W&B syncing is set to `offline`. Starting a new run with run id 2gf88you.


Loaded epoch: 161


In [51]:
from IDS2017_Dataset import IDS2017_Dataset

train_ds =  IDS2017_Dataset(os.path.abspath(params["DIRECTORIES"]["train"]), int(params['HYPERPARAMETERS']['window']))
eval_ds = IDS2017_Dataset(os.path.abspath(params["DIRECTORIES"]["validation"]), int(params['HYPERPARAMETERS']['window']))

In [52]:
from MyDataLoader import MyDataLoader
from torch.utils.data import DataLoader
loader_params = {'batch_size': int(params['HYPERPARAMETERS']['batch_size'])}
training_generator = DataLoader(train_ds, **loader_params)
evaluating_generator = DataLoader(eval_ds, **loader_params)


In [53]:
result = model.evaluate(MyDataLoader(evaluating_generator))


████████████████████████████████████████████████████████████████████████████████████████████████| 100% | 5.56 batch/s | 00:59 | loss: 0.09    

     Epoch stats:    38.41
             EVAL  ----- Loss: 0.094 - Accuracy: 0.99008 - Weighted F1: 0.98977 - Macro F1: 0.67460
                 Labels (pred/true) +++++ SSH-Patator: 1415/1389 - DoS GoldenEye: 1861/2084 - PortScan: 30551/30555 - DoS Slowhttptest: 682/936 - Web Attack  Brute Force: 506/401 - Bot: 655/382 - DDoS: 24936/24969 - DoS slowloris: 1477/1615 - FTP-Patator: 982/968 - DoS Hulk: 46360/46538 - BENIGN: 80175/79646


In [56]:
dictKeys = ['loss', 'categorical_accuracy', 'specificity_at_sensitivity', 'rec_0', 'pre_0', 'rec_1', 'pre_1', 'rec_2', 'pre_2', 'rec_3', 'pre_3', 'rec_4', 'pre_4', 'rec_5', 'pre_5', 'rec_6', 'pre_6', 'rec_7', 'pre_7', 'rec_8', 'pre_8', 'rec_9', 'pre_9', 'rec_10', 'pre_10', 'rec_11', 'pre_11', 'rec_12', 'pre_12', 'rec_13', 'pre_13', 'rec_14', 'pre_14', 'macro_F1', 'weighted_F1']
res = dict(zip(dictKeys, result))

In [57]:
if params['RUN_CONFIG']['dataset'].endswith('LABELS_IDS2017'):
    attack_names = ['SSH-Patator', 'DoS GoldenEye', 'PortScan', 'DoS Slowhttptest', 'Web Attack  Brute Force', 'Bot', 'Web Attack  Sql Injection', 
                    'Web Attack  XSS', 'Infiltration', 'DDoS', 'DoS slowloris', 'Heartbleed', 'FTP-Patator', 'DoS Hulk','BENIGN']
elif params['RUN_CONFIG']['dataset'].endswith('LABELS_IDS2018'):
    attack_names = ['FTP-BruteForce', 'SSH-BruteForce', 'DoS-GoldenEye', 'DoS-Slowloris', 'DoS-SlowHTTPTest', 'DoS-Hulk', 'DDoS attacks-LOIC-HTTP', 
                    'DDoS-LOIC-UDP', 'DDOS-HOIC', 'Brute Force -Web', 'Brute Force -XSS', 'SQL Injection', 'Infiltration', 'Bot', 'BENIGN']
indices = range(len(attack_names))
zip_iterator = zip(attack_names,indices)
attacks_dict = dict(zip_iterator)

In [60]:
accuracies = dict()
precision = dict()
recall = dict()
f1 = dict()
for i in range(len(result["class_accuracy"])):
    accuracies[attack_names[i]] =  result["class_accuracy"][i]
for i in range(len(result["class_precision"])):
    precision[attack_names[i]] =  result["class_precision"][i]
for i in range(len(result["class_recall"])):
    recall[attack_names[i]] =  result["class_recall"][i]
for i in range(len(result["class_f1"])):
    f1[attack_names[i]] =  result["class_f1"][i]

In [61]:
accuracies

{'SSH-Patator': 0.9985601151907847,
 'DoS GoldenEye': 0.8843570057581573,
 'PortScan': 0.9996727213222059,
 'DoS Slowhttptest': 0.6549145299145299,
 'Web Attack  Brute Force': 0.9900249376558603,
 'Bot': 0.9790575916230366,
 'Web Attack  Sql Injection': 0.0,
 'Web Attack  XSS': 0.0,
 'Infiltration': 0.0,
 'DDoS': 0.9966758780888302,
 'DoS slowloris': 0.8352941176470589,
 'Heartbleed': 0.0,
 'FTP-Patator': 0.9927685950413223,
 'DoS Hulk': 0.99333877691349,
 'BENIGN': 0.9935966652437034}

In [131]:
def csvReport(data):
    attacks = dict()
    counter = 0
    first = True
    for row in data:
        if first:
            first = False
            continue
        counter+=1
        label = row[-1]
        if not label in attacks.keys():
            attacks[label] = 0

        attacks[label]+=1
            
    print("     Number of examples in classes: ", attacks)
    print("     %% of examples in classes: ",[(x,y) for x,y in  zip(attacks.keys(), [np.round(el/counter*100,2) for el in attacks.values()])])

In [132]:
import csv

In [133]:
params[directory]["validation_IDS2017"]+"/dataset.csv"

KeyError: 'validation_IDS2017'

In [ ]:

file = os.path.abspath(params[directory]["validation_IDS2017"]+"\\dataset.csv")
with open(file, 'r', newline='') as csvfile:
    data = csv.reader(csvfile, delimiter=',', quotechar='|')
    csvReport(data)

     Number of examples in classes:  {'SSH-Patator': 33598, 'BENIGN': 737954, 'DDoS': 546359, 'DoS Hulk': 298309, 'DoS Slowhttptest': 22664, 'DoS GoldenEye': 7860, 'Web Attack  Brute Force': 238, 'Web Attack  XSS': 82, 'Bot': 39909, 'FTP-Patator': 38195, 'Infiltration': 32094, 'DoS slowloris': 2257, 'Web Attack  Sql Injection': 80}
     %% of examples in classes:  [('SSH-Patator', 1.91), ('BENIGN', 41.94), ('DDoS', 31.05), ('DoS Hulk', 16.95), ('DoS Slowhttptest', 1.29), ('DoS GoldenEye', 0.45), ('Web Attack  Brute Force', 0.01), ('Web Attack  XSS', 0.0), ('Bot', 2.27), ('FTP-Patator', 2.17), ('Infiltration', 1.82), ('DoS slowloris', 0.13), ('Web Attack  Sql Injection', 0.0)]


In [ ]:
output_path = os.path.abspath(params[directory]["validation_IDS2017"]+"\\dataset_short.csv")

In [ ]:
import generator
import importlib
importlib.reload(generator)

<module 'generator' from 'd:\\TFG\\GNN-NIDS-v2\\GNN_NIDS_tensorflow\\generator.py'>

In [ ]:
import pandas as pd

df = pd.read_csv(output_path, header=None, names=generator.features)
df = df[df.columns[0:len(df.columns)-1]]

In [ ]:
pred_dataset = generator.input_fn_predict(os.path.abspath(params[directory]["validation_IDS2017"]+"\\dataset_short.csv"))

In [ ]:
preds = model.predict(val_dataset, batch_size=16)

In [ ]:
generator.attack_names[np.argmax(preds[0])]

'BENIGN'

In [ ]:
preds_str = [row for row in preds]

In [ ]:
preds_str = [generator.attack_names[np.argmax(row)] for row in preds]

In [ ]:
generator.attack_names

['SSH-Patator',
 'DoS GoldenEye',
 'PortScan',
 'DoS Slowhttptest',
 'Web Attack  Brute Force',
 'Bot',
 'Web Attack  Sql Injection',
 'Web Attack  XSS',
 'Infiltration',
 'DDoS',
 'DoS slowloris',
 'Heartbleed',
 'FTP-Patator',
 'DoS Hulk',
 'BENIGN']

In [ ]:
len(preds_str)

1753590

In [ ]:
[(i, el) for i, el in enumerate(preds_str) if el != 'BENIGN']

[(1183, 'DoS Slowhttptest'),
 (2515, 'DoS Slowhttptest'),
 (2977, 'DoS slowloris'),
 (3575, 'DoS slowloris'),
 (3703, 'SSH-Patator'),
 (3740, 'DoS Slowhttptest'),
 (3762, 'DoS Slowhttptest'),
 (5374, 'SSH-Patator'),
 (5379, 'SSH-Patator'),
 (5382, 'SSH-Patator'),
 (5383, 'SSH-Patator'),
 (5386, 'SSH-Patator'),
 (5387, 'SSH-Patator'),
 (5395, 'SSH-Patator'),
 (5396, 'SSH-Patator'),
 (5397, 'SSH-Patator'),
 (5400, 'SSH-Patator'),
 (5401, 'SSH-Patator'),
 (5404, 'SSH-Patator'),
 (5405, 'SSH-Patator'),
 (5407, 'SSH-Patator'),
 (5410, 'SSH-Patator'),
 (5415, 'SSH-Patator'),
 (5417, 'SSH-Patator'),
 (5418, 'SSH-Patator'),
 (5419, 'SSH-Patator'),
 (5420, 'SSH-Patator'),
 (5421, 'SSH-Patator'),
 (5422, 'SSH-Patator'),
 (5423, 'SSH-Patator'),
 (5427, 'SSH-Patator'),
 (5428, 'SSH-Patator'),
 (5430, 'SSH-Patator'),
 (5431, 'SSH-Patator'),
 (5432, 'SSH-Patator'),
 (5433, 'SSH-Patator'),
 (5434, 'SSH-Patator'),
 (5435, 'SSH-Patator'),
 (5436, 'SSH-Patator'),
 (5439, 'SSH-Patator'),
 (5440, 'SSH-Pat